In [1]:
from aut import *
from pyspark.sql.functions import col, explode_outer

In [2]:
webpages = WebArchive(sc, sqlContext, "/home/nruest/Projects/au/sample-data/geocities") \
  .all() \
  .filter("crawl_date is not NULL")\
  .filter(~(col("url").rlike(".*robots\\.txt$")) & (col("mime_type_web_server").rlike("text/html") | col("mime_type_web_server").rlike("application/xhtml+xml") | col("url").rlike("(?i).*htm$") | col("url").rlike("(?i).*html$")))\
  .filter(col("http_status_code") == 200)\
  .select("domain", "url", "crawl_date", explode_outer(extract_links("url", "raw_content")).alias("link")) \
  .filter(col("raw_content").like("%food%")) \
  .select("url", "domain", "crawl_date", col("link._1").alias("destination_page"))

In [3]:
webpages.show(5, False)

[2022-05-29T22:12:01.819Z - 00000 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143512-00103-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


22/05/29 18:12:02 WARN PDFParser: J2KImageReader not loaded. JPEG2000 files will not be processed.
See https://pdfbox.apache.org/2.0/dependencies.html#jai-image-io
for optional dependencies.

22/05/29 18:12:02 WARN TesseractOCRParser: Tesseract OCR is installed and will be automatically applied to image files unless
you've excluded the TesseractOCRParser from the default parser.
Tesseract may dramatically slow down content extraction (TIKA-2359).
As of Tika 1.15 (and prior versions), Tesseract is automatically called.
In future versions of Tika, users may need to turn the TesseractOCRParser on via TikaConfig.
22/05/29 18:12:02 WARN SQLite3Parser: org.xerial's sqlite-jdbc is not loaded.
Please provide the jar on your classpath to parse sqlite files.
See tika-parsers/pom.xml for the correct version.


+-------------------------------------------------------------------------------------+-------------+--------------+--------------------------------------------+
|url                                                                                  |domain       |crawl_date    |destination_page                            |
+-------------------------------------------------------------------------------------+-------------+--------------+--------------------------------------------+
|http://geocities.com/kelsoonbutler/boardDocs/BoardMeetingMinutes_AGM_March12_2002.htm|geocities.com|20091027143512|null                                        |
|http://geocities.com/krooyaimaha/health5.htm                                         |geocities.com|20091027143546|http://geocities.com/krooyaimaha/health5.htm|
|http://geocities.com/krooyaimaha/health5.htm                                         |geocities.com|20091027143546|http://geocities.com/krooyaimaha/health5.htm|
|http://geocities.com/krooya